# PyNearestNeighbours Evaluatin

We have been using either faiss or skelar nto find joins. 

How good is https://pynndescent.readthedocs.io/en/latest/index.html ?

In [7]:
ERA_file = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/ERA_timevariable/ERA_2016_01.grib'
MODIS_file = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/MODIS/aquaDay_errorGTE03K_04km_2016-01-16.tif'

In [12]:
import xarray as xr


In [9]:
ERA_month = xr.open_dataset(ERA_file,engine='cfgrib',backend_kwargs={'indexpath': ''})


In [18]:
timestamps = pd.to_datetime(ERA_month.time) 


In [22]:
t = timestamps[368] #iterate over every time (hour)

In [13]:
MODIS_data

<xarray.DataArray 'band_data' (band: 1, y: 5400, x: 10800)>
[58320000 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.9 -179.9 ... 179.9 180.0 180.0
  * y            (y) float64 89.98 89.95 89.92 89.88 ... -89.92 -89.95 -89.98
    spatial_ref  int64 ...
Attributes:
    long_name:  LST_Day_1km

In [16]:
#Prep MODIS data
import numpy as np
import pandas as pd
latitude_bound = 70
date_string = '2016-01-16'
MODIS_data = xr.open_dataarray(MODIS_file,engine="rasterio")


#Make some edits to file
MODIS_data = MODIS_data.rename({'x':'longitude','y':'latitude'})

#Filter by latitude bound
space_filter = np.expand_dims(np.abs(MODIS_data.latitude) < latitude_bound,axis=(0,-1))
mask = np.logical_and(np.isfinite(MODIS_data),space_filter) #make it a 2d mask
MODIS_data = MODIS_data.where(mask,drop=True)

#Convert local satellite time to UTC and round to nearest hour
time_delta = pd.to_timedelta(MODIS_data.longitude.data/15,unit='H') 
time_UTC = (pd.to_datetime([date_string + " " + "13:30"]*time_delta.shape[0]) - time_delta).round('H')









In [17]:
time_UTC

DatetimeIndex(['2016-01-17 01:00:00', '2016-01-17 01:00:00',
               '2016-01-17 01:00:00', '2016-01-17 01:00:00',
               '2016-01-17 01:00:00', '2016-01-17 01:00:00',
               '2016-01-17 01:00:00', '2016-01-17 01:00:00',
               '2016-01-17 01:00:00', '2016-01-17 01:00:00',
               ...
               '2016-01-16 02:00:00', '2016-01-16 02:00:00',
               '2016-01-16 02:00:00', '2016-01-16 02:00:00',
               '2016-01-16 02:00:00', '2016-01-16 02:00:00',
               '2016-01-16 02:00:00', '2016-01-16 02:00:00',
               '2016-01-16 02:00:00', '2016-01-16 02:00:00'],
              dtype='datetime64[ns]', length=10779, freq=None)

In [23]:
#Filter to only select the hour of data we want
time_filter = np.expand_dims(time_UTC == t,axis=(0,1))
mask = np.logical_and(np.isfinite(MODIS_data),time_filter)
MODIS_hour = MODIS_data.where(mask,drop=True).load() 
MODIS_df = MODIS_hour.to_dataframe(name='MODIS_LST').reset_index().dropna() #Make everything a pandas df to pass into faiss_knn. Unnecessary step?


In [26]:
delta = 1.0 # Enveloping box
bounds = {"latitude_min" : MODIS_df.latitude.min()-delta,
        "latitude_max" :   MODIS_df.latitude.max()+delta,
        "longitude_min":   MODIS_df.longitude.min()-delta,
        "longitude_max":   MODIS_df.longitude.max()+delta
}

In [27]:
#Filter month of ERA data to an hour
time_filter = (ERA_month.time == t)
ERA_hour = ERA_month.where(time_filter,drop=True)

#And covert longitude to long1
ERA_hour = ERA_hour.assign_coords({"longitude": (((ERA_hour.longitude + 180) % 360) - 180)})

# Also filter by latitude/longtiude
longitude_filter = (ERA_hour.longitude > bounds['longitude_min']) & (ERA_hour.longitude < bounds['longitude_max'])
latitude_filter =  (ERA_hour.latitude > bounds['latitude_min']) & (ERA_hour.latitude < bounds['latitude_max'])
ERA_snap = ERA_hour.where(longitude_filter & latitude_filter,drop=True)



In [29]:
ERA_df = ERA_snap.to_dataframe().reset_index()                # Make it a df


In [30]:
import pynndescent

def match(database,query):

    #Construct NN     
    NN = NearestNeighbors(n_neighbors=1, algorithm='ball_tree', leaf_size=60,metric='haversine') #algorithm = balltree, kdtree or brutie force


    NN.fit(np.deg2rad(database[['latitude', 'longitude']].values))

    query_lats = query['latitude'].astype(np.float64)
    query_lons = query['longitude'].astype(np.float64)


    X = np.deg2rad(np.c_[query_lats, query_lons])
    distances, indices = NN.kneighbors(X, return_distance=True)


    r_km = 6371 # multiplier to convert to km (from unit distance)
    distances = distances*r_km

    df = query.reset_index().join(database.iloc[indices.flatten()].reset_index(), lsuffix='_MODIS',rsuffix='_ERA')
    df['H_distance'] = distances

    #Filter out any large distances
    tolerance = 50 #km
    df_filtered = df.query('H_distance < %.9f' % tolerance)

    #Group it. Each ERA point has a bunch of MODIS points. Group and average
    df_grouped = df_filtered.groupby(['latitude_ERA','longitude_ERA'],as_index=False).mean()

    return df_grouped


In [200]:
%%time

#NEW METHOD

index = pynndescent.NNDescent(np.deg2rad(ERA_df[['latitude', 'longitude']].values),metric='haversine',n_jobs=5)
indices_new, distances_new = index.query(np.deg2rad(MODIS_df[['latitude', 'longitude']].values),k=1)
df_new = MODIS_df.reset_index().join(ERA_df.iloc[indices_new.flatten()].reset_index(), lsuffix='_MODIS',rsuffix='_ERA')


CPU times: user 17 s, sys: 2.39 s, total: 19.4 s
Wall time: 10.2 s


In [201]:
%%time  

#OLD METHOD

NN = NearestNeighbors(n_neighbors=1, algorithm='ball_tree', leaf_size=60,metric='haversine') #algorithm = balltree, kdtree or brutie force
NN.fit(np.deg2rad(ERA_df[['latitude', 'longitude']].values))
distances_old, indices_old = NN.kneighbors(np.deg2rad(MODIS_df[['latitude', 'longitude']].values), return_distance=True)
df_old = MODIS_df.reset_index().join(ERA_df.iloc[indices_old.flatten()].reset_index(), lsuffix='_MODIS',rsuffix='_ERA')


CPU times: user 14.5 s, sys: 1.55 ms, total: 14.5 s
Wall time: 14.5 s


In [202]:
dfnew_xt = df_new[['latitude_MODIS','longitude_MODIS','latitude_ERA','longitude_ERA']].copy()
dfold_xt = df_old[['latitude_MODIS','longitude_MODIS','latitude_ERA','longitude_ERA']].copy()

dfnew_xt = dfnew_xt.add_suffix('_new')
dfold_xt = dfold_xt.add_suffix('_old')
df_c = pd.concat([dfnew_xt,dfold_xt],axis=1)
df_c['diff_latitude'] = df_c['latitude_ERA_new'] -  df_c['latitude_ERA_old']
df_c['diff_longitude'] = df_c['longitude_ERA_new'] -  df_c['longitude_ERA_old']



In [203]:
def haver(lat1_deg,lon1_deg,lat2_deg,lon2_deg):

    """
    Given coordinates of two points IN DEGREES calculate the haversine distance
    """

    #Convert degrees to radians
    lat1 = np.deg2rad(lat1_deg)
    lon1 = np.deg2rad(lon1_deg)
    lat2 = np.deg2rad(lat2_deg)
    lon2 = np.deg2rad(lon2_deg)


    #...and the calculation
    delta_lat = lat1 -lat2
    delta_lon = lon1 -lon2
    Re = 6371 #km
    Z = np.sin(delta_lat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(delta_lon/2)**2
    H = 2*Re*np.arcsin(np.sqrt(Z)) #Haversine distance in km
    return H


In [204]:
df_c['haver_old'] = haver(df_c['latitude_MODIS_old'],df_c['longitude_MODIS_old'],df_c['latitude_ERA_old'],df_c['longitude_ERA_old']) #Haversine distance
df_c['haver_new'] = haver(df_c['latitude_MODIS_new'],df_c['longitude_MODIS_new'],df_c['latitude_ERA_new'],df_c['longitude_ERA_new']) #Haversine distance


In [205]:
df_q = df_c.query("diff_latitude != 0.0 | diff_longitude != 0.0").copy()

In [206]:
def is_better(old_method,new_method):
    
    if old_method< new_method:
        return "Old method is better"
    if old_method > new_method:
        return "New method is better"

df_q['Q'] = df_q.apply(lambda row: is_better(row['haver_old'], row['haver_new']), axis=1)


In [207]:
df_q

,latitude_MODIS_new,longitude_MODIS_new,latitude_ERA_new,longitude_ERA_new,latitude_MODIS_old,longitude_MODIS_old,latitude_ERA_old,longitude_ERA_old,diff_latitude,diff_longitude,haver_old,haver_new,Q
11876,68.883333,78.950000,68.711899,79.259259,68.883333,78.950000,68.992929,78.750000,-0.28103,0.509259,14.573393,22.760879,Old method is better
11921,68.883333,80.450000,68.711899,80.740741,68.883333,80.450000,68.992929,80.250000,-0.28103,0.490741,14.573393,22.362657,Old method is better
11922,68.883333,80.483333,68.711899,80.740741,68.883333,80.483333,68.992929,80.250000,-0.28103,0.490741,15.344360,21.691914,Old method is better
11966,68.883333,81.950000,68.711899,82.222222,68.883333,81.950000,68.992929,81.750000,-0.28103,0.472222,14.573393,21.982458,Old method is better
11967,68.883333,81.983333,68.711899,82.222222,68.883333,81.983333,68.992929,81.750000,-0.28103,0.472222,15.344360,21.346562,Old method is better
...,...,...,...,...,...,...,...,...,...,...,...,...,...
433849,20.750000,86.116667,20.936761,86.222222,20.750000,86.116667,20.655731,86.062500,0.28103,0.159722,11.900434,23.485882,Old method is better
476396,-68.816667,81.316667,-68.992929,81.000000,-68.816667,81.316667,-68.711899,81.481481,-0.28103,-0.481481,13.408079,23.339870,Old method is better
476450,-68.850000,81.250000,-68.992929,81.000000,-68.850000,81.250000,-68.711899,81.481481,-0.28103,-0.481481,17.961024,18.776036,Old method is better
476451,-68.850000,81.316667,-68.992929,81.000000,-68.850000,81.316667,-68.711899,81.481481,-0.28103,-0.481481,16.727440,20.321313,Old method is better


In [208]:
df_q.groupby(['Q']).count()

,latitude_MODIS_new,longitude_MODIS_new,latitude_ERA_new,longitude_ERA_new,latitude_MODIS_old,longitude_MODIS_old,latitude_ERA_old,longitude_ERA_old,diff_latitude,diff_longitude,haver_old,haver_new
Q,,,,,,,,,,,,
Old method is better,2472,2472,2472,2472,2472,2472,2472,2472,2472,2472,2472,2472
